## Parameters

In [8]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
np.random.seed(0)

steps = 15
dt = 0.1
grid_size = (1, 1)
N = 32
M = 32
dn = grid_size[0] / N
dm = grid_size[1] / M

dim = (N, M, 2)
initial_u = np.random.sample(size=dim)
nu = 1
rho = 1
p = np.zeros(dim)


## Definitions

In [9]:
def compute_convection(u_in):
    convection = np.zeros_like(u_in)

    # Adapt this function if you use a mesh with non uniform spacing
    def distance_1d(loc1, loc2):
        return sum(abs(loc1 - loc2) * (grid_size[0] / N, grid_size[1] / M))
    
    def derivative(loc, axis, component):
        prev_pt = np.clip(loc - axis, 0, [N - 1, M - 1])
        next_pt = np.clip(loc + axis, 0, [N - 1, M - 1])
        return (u_in[*next_pt, component] - u_in[*prev_pt, component]) / distance_1d(next_pt, prev_pt)
        
    
    for n in range(N):
        for m in range(M):
            grads = np.array([
                [derivative(np.array([n, m]), np.array([1, 0]), 0), derivative(np.array([n, m]), np.array([1, 0]), 1)], 
                [derivative(np.array([n, m]), np.array([0, 1]), 0), derivative(np.array([n, m]), np.array([0, 1]), 1)]])
            convection[n, m] = np.dot(u_in[n, m], grads)
            
    return convection

In [10]:
def compute_viscous_drag(u_in):
    
    viscous_drag = np.zeros_like(u_in)
    
    def distance_1d(loc1, loc2):
        return sum(abs(loc1 - loc2) * (grid_size[0] / N, grid_size[1] / M))
        
    def derivative_2nd(loc, axis, component):
        prev_pt = np.clip(loc - axis, [0, 0], [N - 1, M - 1])
        next_pt = np.clip(loc + axis, [0, 0], [N - 1, M - 1])
        return (u_in[*next_pt, component] - (2 * u_in[*loc, component]) - u_in[*prev_pt, component]) / distance_1d(next_pt, prev_pt) ** 2
    
    for n in range(N):
        for m in range(M):
            for component in range(2):
                viscous_drag[n, m, component] = derivative_2nd(np.array([n, m]), np.array([2, 0]), component) + derivative_2nd(np.array([n, m]), np.array([0, 2]), component)
    return viscous_drag * nu

In [11]:
# (N,M,1) -> (N,M,2)
def compute_pressure(pressure_in):
    pressure_out = np.zeros_like(initial_u)
    
    def distance_1d(loc1, loc2):
        return sum(abs(loc1 - loc2) * (1 / N, 1 / M))
    
    def derivative(loc, axis):
        prev_pt = np.clip(loc - axis, 0, [N - 1, M - 1])
        next_pt = np.clip(loc + axis, 0, [N - 1, M - 1])
        return (pressure_in[*next_pt, 0] - pressure_in[*prev_pt, 0]) / distance_1d(next_pt, prev_pt)
        
    for n in range(N):
        for m in range(M):
            pressure_out[n, m, 0] = derivative(np.array([n, m]), np.array([1, 0]))
            pressure_out[n, m, 1] = derivative(np.array([n, m]), np.array([0, 1]))
    
    return (1 / rho) * pressure_out


In [12]:
def compute_updated_pressure(p_in, inter_u_in):
    p2 = np.zeros_like(p_in)
    
    def fetch(source, mock):
        def fetch_intern(n, m):
            if n < 0 or n >= N:
                return mock
            if m < 0 or m >= M:
                return mock
            return source[n, m]
        return fetch_intern
    
    fetch_p = fetch(p_in, 0)
    fetch_u = fetch(inter_u_in, 0)
    
    for n in range(N):
        for m in range(M):
            rnm = rho / dt * ((fetch_u(n + 1, m) - fetch_u(n - 1, m)) / (2 * dn) + (fetch_u(n, m + 1) - fetch_u(n, m - 1)) / (2 * dm) )
            x = (fetch_p(n + 2, m) - fetch_p(n - 2, m)) * 4 * dm ** 2 + (fetch_p(n, m + 2) - fetch_p(n, m - 2)) * 4 * dn ** 2
            x -= rnm * 16 * dn ** 2 * dm ** 2
            x /= 8 * (dn ** 2 + dm ** 2)
            p2[n, m] = x

    return compute_pressure(p2)


## Simulation run

In [6]:
all_u = np.zeros((steps, N, M, 2))
all_u[0] = initial_u
for step in tqdm(range(1, steps)):
    u_update = -compute_convection(all_u[step - 1]) + compute_viscous_drag(all_u[step - 1])
    inter_u = u_update * dt
    p = compute_updated_pressure(p, inter_u)
    all_u[step] = all_u[step - 1] - p

 50%|██████████████████████████████████████████                                          | 7/14 [00:01<00:01,  5.04it/s]/tmp/ipykernel_2420/319527876.py:19: RuntimeWarning: overflow encountered in dot
  convection[n, m] = np.dot(u_in[n, m], grads)
/tmp/ipykernel_2420/319527876.py:19: RuntimeWarning: invalid value encountered in dot
  convection[n, m] = np.dot(u_in[n, m], grads)
/tmp/ipykernel_2420/937616057.py:21: RuntimeWarning: invalid value encountered in subtract
  rnm = rho / dt * ((fetch_u(n + 1, m) - fetch_u(n - 1, m)) / (2 * dn) + (fetch_u(n, m + 1) - fetch_u(n, m - 1)) / (2 * dm) )
/tmp/ipykernel_2420/937616057.py:21: RuntimeWarning: invalid value encountered in add
  rnm = rho / dt * ((fetch_u(n + 1, m) - fetch_u(n - 1, m)) / (2 * dn) + (fetch_u(n, m + 1) - fetch_u(n, m - 1)) / (2 * dm) )
100%|███████████████████████████████████████████████████████████████████████████████████| 14/14 [00:02<00:00,  4.87it/s]


## Result visualization

In [7]:
%matplotlib inline
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt


def update(step = (0, steps, 1)):
    plt.imshow(np.linalg.norm(all_u[step], axis=2))
    plt.show()

interact(update)

interactive(children=(IntSlider(value=7, description='step', max=15), Output()), _dom_classes=('widget-interac…

<function __main__.update(step=(0, 15, 1))>

# Comment

I observe that after 7 steps, the velocities are infested with nan values.

Let's check if the continuity equation is respected: $$ \nabla \cdot \mathbf{u} = 0$$

In [17]:
def compute_divergence(u_in):
    divergence = np.zeros((N, M))
    bc = 0
    def fetch(n, m, c):
        if n < 0 or n >= N or m < 0 or m >= M:
            return bc
        return u_in[n, m, c]
        
    for n in range(N):
        for m in range(M):
            divergence[n, m] = (fetch(n + 1, m, 0) - fetch(n - 1, m, 0)) / (2 * dn) + (fetch(n, m + 1, 1) - fetch(n, m - 1, 1)) / (2 * dn)
    return divergence

In [21]:


def update(step = (0, steps, 1)):
    div = compute_divergence(all_u[step])
    plt.imshow(div)
    plt.colorbar()
    plt.show()

interact(update)

interactive(children=(IntSlider(value=7, description='step', max=15), Output()), _dom_classes=('widget-interac…

<function __main__.update(step=(0, 15, 1))>

We observe that divergence reach very high values.
Let's try with smaller delta time.

For covenience, I will bundle the simulation nicely.

In [30]:
from dataclasses import dataclass

%matplotlib inline
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

@dataclass
class CFDResult:
    all_u: np.array

    def plot_velocities(self):
        def update(step = (0, steps, 1)):
            plt.imshow(np.linalg.norm(all_u[step], axis=2))
            plt.show()
        return interact(update)

    def plot_divergences(self):
        def update(step = (0, steps, 1)):
            div = compute_divergence(all_u[step])
            plt.imshow(div)
            plt.colorbar()
            plt.show()
        return interact(update)

@dataclass
class CFDSimulation:
    steps = 15
    dt = 0.1
    grid_size = (1, 1)
    N = 32
    M = 32
    dn = grid_size[0] / N
    dm = grid_size[1] / M
    dim = (N, M, 2)
    initial_u = np.random.sample(size=dim)
    nu = 1
    rho = 1
    p = np.zeros(dim)

    def launch(self):
        all_u = np.zeros((steps, N, M, 2))
        all_u[0] = initial_u
        for step in tqdm(range(1, steps)):
            u_update = -compute_convection(all_u[step - 1]) + compute_viscous_drag(all_u[step - 1])
            inter_u = u_update * dt
            p = compute_updated_pressure(self.p, inter_u)
            all_u[step] = all_u[step - 1] - self.p
        return CFDResult(all_u)

In [34]:

simu = CFDSimulation()
simu.dt = 0.001
result = simu.launch()
result.plot_velocities()

100%|███████████████████████████████████████████████████████████████████████████████████| 14/14 [00:02<00:00,  4.77it/s]


interactive(children=(IntSlider(value=7, description='step', max=15), Output()), _dom_classes=('widget-interac…

<function __main__.CFDResult.plot_velocities.<locals>.update(step=(0, 15, 1))>

dt was not the issue it seems. My next guess is the BC. Time for a new notebook.